In [7]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
from jpeg import mogrify_degrade, opencv_degrade

URL = 'https://upload.wikimedia.org/wikipedia/commons/d/d3/PSNR-example-base.png'
Qs = [10, 30, 90]

In [8]:
def download(url) -> str:
    """Retrieve the file at url, save it locally and return the path."""
    basename = os.path.basename(url)
    if os.path.exists(basename):
        return basename

    from urllib.request import urlopen
    print('Downloading: {}'.format(basename))

    with urlopen(url) as response, open(basename, 'wb') as output:
        output.write(response.read())

    return basename

def opencv_degrade(orig, filename, q, grayscale=False):
    img = cv2.imread(
        orig,
        cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_UNCHANGED,
    )
    cv2.imwrite(filename, img, [cv2.IMWRITE_JPEG_QUALITY, q])

In [13]:
def load(filename):
    img = Image.open(filename).convert("RGB")
    return img

def generate_images(filename, Qs):
    result = {'orig': load(filename)}
    for Q in Qs:
        out = f"Q{Q}_{os.path.splitext(filename)[0]}.jpg"
        opencv_degrade(filename, out, Q)
        result[Q] = load(out)
    return result

def draw_tiles(rows, hgap=10, vgap=10, captions=None):
    img_w = max(img.width for row in rows for img in row)
    img_h = max(img.height for row in rows for img in row)
    n_rows = len(rows)
    n_cols = max(len(row) for row in rows)
    
    off_w = img_w + hgap
    off_h = img_h + vgap
    canv_w = off_w * n_cols - hgap
    canv_h = off_h * n_rows - (0 if captions else vgap)  # (leaving room for captions)
    canvas = Image.new("RGB", (canv_w, canv_h), color=(255, 255, 255))
    
    
    for r, row in enumerate(rows):
        for c, img in enumerate(row):
            canvas.paste(img, (c * off_w, r * off_h))
            
    if captions:
        d = ImageDraw.Draw(canvas)
        fnt = ImageFont.truetype("Ubuntu-L.ttf", vgap * 2 // 3)
        for r, row in enumerate(captions):
            for c, caption in enumerate(row):
                d.text((off_w * c + hgap, off_h * r + img_h + 1), caption, fill=(0, 0, 0), font=fnt)
    
    return canvas

def ssim(img1: Image, img2: Image, gray=False) -> float:
    if gray:
        img1 = img1.convert("L")
        img2 = img2.convert("L")
    return structural_similarity(np.array(img1), np.array(img2), multichannel=not gray)

def psnr(img1, img2, gray=False):
    if gray:
        img1 = img1.convert("L")
        img2 = img2.convert("L")
    return peak_signal_noise_ratio(np.array(img1), np.array(img2))

In [14]:
orig = generate_images(ORIG, Qs)

In [15]:
for q in Qs:
    data = {
        'Q': q,
        'PSNR': psnr(orig['orig'], orig[q]),
        'SSIM': ssim(orig['orig'], orig[q]),
        'PSNR_L': psnr(orig['orig'], orig[q], True),
        'SSIM_L': ssim(orig['orig'], orig[q], True),
    }
    print(data)

{'Q': 10, 'PSNR': 28.77028635497755, 'SSIM': 0.8516931177017346, 'PSNR_L': 31.448226433237302, 'SSIM_L': 0.8799162142539958}
{'Q': 30, 'PSNR': 33.13845605784299, 'SSIM': 0.9217806333971129, 'PSNR_L': 36.80667955166063, 'SSIM_L': 0.9483089882373851}
{'Q': 90, 'PSNR': 38.58898392271544, 'SSIM': 0.9687070837089792, 'PSNR_L': 45.612177609530605, 'SSIM_L': 0.9888889939803718}


The PSNR_L (PSNR of Luma) matches the data from the Wikipedia example.